In [48]:
#self attention realize
import math
import torch
import torch.nn as nn
import warnings as ws
ws.filterwarnings("ignore")

In [49]:
class SelfAttV3(nn.Module):
    def __init__(self,hidden_dim,bias = True,drop_rate=0.1,refect_matrix=False):
        super(SelfAttV3,self).__init__()
        self.hiddem_dim = hidden_dim
        self.bias = bias
        self.qkv = nn.Linear(hidden_dim,hidden_dim*3,bias=self.bias)
        
        self.dropout = nn.Dropout(drop_rate)
        if refect_matrix:
            self.outputs = nn.Linear(hidden_dim,hidden_dim)
        else:
            self.outputs = None
    def forward(self,x,attention_mask=None):
        QKV =self.qkv(x)
        q,k,v = torch.split(QKV,self.hiddem_dim,dim=-1)
        attention_score =q@k.transpose(-1,-2)/math.sqrt(self.hiddem_dim)
        #在计算softmax之前对矩阵进行mask，将mask为0的元素置为负无穷，使计算softmax时，mask为0的地方权重接近0
        if attention_mask is not None:
            attention_score = attention_score.masked_fill(attention_mask==0,float('-inf'))
        attention_weight = torch.softmax(attention_score,dim=-1)
        attention_weight = self.dropout(attention_weight)
        attention_weight = attention_weight@v
        if self.outputs is not None:
            outputs = self.outputs(attention_weight)
        else:
            outputs = attention_weight
        return outputs

In [ ]:
class SelfAttV4(nn.Module):
    def __init__(self,hidden_dim,bias = True,drop_rate=0.1,refect_matrix=False):
        super(SelfAttV4,self).__init__()      

In [50]:
def repeat_test(mask):
    print("maske.shape:",mask.shape)
    print('mask:\n',mask)
    after_mask = mask.unsqueeze(1)#.repeat(1,x.shape[1],1)
    # selfatt = SelfAttV1(x.shape[-1])
    # print(selfatt(x).shape) 
    print('repeat after:',after_mask.shape) 
    print(after_mask)     
    after_mask2 = mask.unsqueeze(-1)#.repeat(1,1,x.shape[1])
    print('repeat after2:',after_mask2.shape) 
    print(after_mask2)

In [51]:

if __name__== '__main__':
    x = torch.randn(1,3,4)
    mask = torch.tensor([[1,1,0],[1,1,1],[1,1,1]])
    
    print("maske.shape:",mask.shape)
    print('mask:\n',mask)
    after_mask = mask.unsqueeze(1).repeat(1,x.shape[1],1)
    model = SelfAttV3(x.shape[-1])
 
    model_outputs=model(x,after_mask)
    print("model_outputs.shape：\n",model_outputs.shape)
    print("-"*30)
    repeat_test(mask)

maske.shape: torch.Size([3, 3])
mask:
 tensor([[1, 1, 0],
        [1, 1, 1],
        [1, 1, 1]])
model_outputs.shape：
 torch.Size([3, 3, 4])
------------------------------
maske.shape: torch.Size([3, 3])
mask:
 tensor([[1, 1, 0],
        [1, 1, 1],
        [1, 1, 1]])
repeat after: torch.Size([3, 1, 3])
tensor([[[1, 1, 0]],

        [[1, 1, 1]],

        [[1, 1, 1]]])
repeat after2: torch.Size([3, 3, 1])
tensor([[[1],
         [1],
         [0]],

        [[1],
         [1],
         [1]],

        [[1],
         [1],
         [1]]])
